# Ormir-mids: dcm2omids 

- By: [Simone Poncioni](https://github.com/simoneponcioni), [Serena Bonaretti](https://sbonaretti.github.io/)
- Notebook created by merging and modifying a notebook by Leonardo Barzaghi, [Donnie Cameron](https://github.com/DC-3T), Judith Cueto Fernandez, Jilmen Quintiens, [Francesco Santini](https://github.com/fsantini), and a notebook by [Gianluca Iori](https://github.com/gianthk) and Francesco Chiumento
- Code license: Apache 2.0
- Narrative license: CC-BY-NC-SA
- How to cite: Sarah Manske, Mahdi Hosseinitabatabaei, Pholpat Durongbhan, Michael Kuczynski, Simone Poncioni, Gianluca Iori, Serena Bonaretti. *Why and how to share MSK imaging data?* Worskhop at the 24th International Workshop on Quantitative Musculoskeletal Imaging (QMSKI). The Barossa Valley, South Australia. November 3-8, 2024.

---

### Aims

The [ORMIR-MIDS package](https://github.com/ormir-mids/ormir-mids/tree/Jupyter) converts musculoskeletal DICOM images into a data structure inspired by [BIDS](https://bids.neuroimaging.io/)[<sup id="fn1-back">1</sup>](#fn1), following the [ORMIR-MIDS specs](https://ormir-mids.github.io/specs.html)

In this notebook you will learn how to:

1. Convert images to ORMIR-MIDS data structure 
2. Explore an ORMIR-MIDS data structure
3. Explore an image volume (before conversion)
4. Visualize the image volume in jupyter notebook

---

### Installations

- Let's install ORMIR-MIDS:

In [ ]:
%pip install ormir-mids

# Alternatively, you can install the package from the source code
# !git clone https://github.com/ormir-mids/ormir-mids.git
# %cd ormir-mids
# %pip install -e .

- To explore the folder and data structure, we will need the [*Directory Tree*](`https://pypi.org/project/directory-tree/`) package:

In [ ]:
# to explore the folder and data structure
%pip install directory_tree
# to visualize images in a jupyter notebook
%pip install k3d
# to convert to SimpleITK
%pip install SimpleITK
# print out our machine characteristics and the version of the packages (for reproducibility)
%pip install watermark

Or if you use conda:
> `conda install -c conda-forge directory-tree k3d watermark`

> `conda install -c https://conda.anaconda.org/simpleitk SimpleITK`

### Imports 

- We will need to use the following packages:

In [43]:
import ormir_mids
from ormir_mids.dcm2omids import convert_dicom_to_ormirmids
from ormir_mids.utils.io import find_omids, load_omids, load_dicom

In [44]:
import zipfile
from pathlib import Path

import requests
import SimpleITK as sitk
import k3d
import numpy as np

from directory_tree import DisplayTree

### Functions

- This is a function to download and unzip the folder containing the image data:

In [45]:
def download_and_unzip(url, extract_to='.'):
    """
    Downloads a ZIP file from the specified URL and extracts it.

    Parameters
    ----------
    url : str
        The URL of the ZIP file to download.
    extract_to : str, optional
        The directory to extract the contents to. Defaults to the current directory.

    Returns
    -------
    None
    """
    
    extract_to_path = Path(extract_to)
    if not extract_to_path.exists():
        extract_to_path.mkdir(parents=True)
    local_zip_path = extract_to_path / 'downloaded.zip'

    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses

    with local_zip_path.open('wb') as file:
        file.write(response.content)

    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_path)

    local_zip_path.unlink()

### Variables

- Here are the variables we will use:

In [46]:
# zipped data folder to download
zipped_folder = "https://github.com/ormir-mids/ormir-mids/raw/main/dicom.zip"
# designated unzipped data folder
dicom_folder = "data"
# folder that will contain the images in ORMIR-MIDS data structure
omids_folder = "omids_data"

---

## Getting the data

- Extract the data from the provided zipped folder and save them in *dicom_folder*:

In [47]:
download_and_unzip(zipped_folder, extract_to=dicom_folder)

- Let's have a look at what data are available. We'll use the Python package `directory_tree`, which allows us to display our directory structure:

In [ ]:
DisplayTree(dirPath=dicom_folder,
            stringRep=False,
            header=True,
            maxDepth=3,
            showHidden=False,
            sortBy=2, # 0 - Default, 1 - Files First, 2 - Directories First
            )

- As you can see, these are one Multi-Echo GRadient-Echo (MEGRE) image from GE and one Multi-Echo Spin-Echo (MESE) image from Philips
- The GE data comprise multiple DICOM files, whereas the Philips data are contained in a single, large file
- You can also browse the files inside their dicom directory from the file explorer

---
# 1. Converting to ORMIR-MIDS data structure

- Let's convert the images from out-of-scanner `.dcm` to the ORMIR-MIDS structure:

In [ ]:
convert_dicom_to_ormirmids(input_folder=dicom_folder+"/dicom/", output_folder=omids_folder, anonymize='anon', recursive=True)

Alternatively, you can run it in the command line as executable

Retrieve documentation:
```sh
dcm2omids -h
```

Anonymize and convert the data:


```sh
# dcm2omids -anonymize <pseudo_name> -recursive <input_dir> <output_dir>
dcm2omids -a mypatient -r data/dicom bids_output
```

- The output data are saved to the new directory called `omids_folder`. Let's explore it:

In [ ]:
DisplayTree(dirPath=omids_folder,
            stringRep=False,
            header=True,
            maxDepth=3,
            showHidden=False,
            sortBy=2, # 0 - Default, 1 - Files First, 2 - Directories First
            )

- The converted data are sorted under `mr-anat` for anatomical images and `mr-quant` for quantitative maps
- Every dataset is composed of:
  -  A `.nii.gz` file containing the image data
  -  A set of `.json` files containing the headers. Specifically:
     - A simple `.json` file containing useful information about the data
     - A `_patient.json` file containing private patient data. *Delete this file if you want to anonymize your data!*
     - A `_extra.json` file containing extra information that can be used to reconstruct the DICOM dataset from the ORMIR-MIDS data, so it can be stored again in PACS



---
# 2. Exploring the ORMIR-MIDS data structure

### 2.1 Finding images
- Let's find the `mese` image in the folder:

In [ ]:
mese_data_list = find_omids(omids_folder, 'mese')
print(mese_data_list)

- *Exercise: Find the `mese_ph` image in the folder:*

### 2.2 Reading an image

In [53]:
omids_mese = load_omids(mese_data_list[0])

### 2.3 Reading `.json` files

- Each image volume has three dictionaries (corresponding to the three `.json` files) associated with it:
  - `omids_header`containing relevant information
  - `patient_header` containing privacy-relevant information (file is missing in case of anonymization)
  - `extra_header`: a collection of the remaining DICOM tags

- Let's read the patient header:

In [ ]:
omids_mese.patient_header

- *Exercise: Read the extra header*:

In [ ]:
omids_mese.extra_header

---
## 3. Exploring `.dcm` images before conversion

### 3.1 Load DICOM image, header, and metadata

- ORMIR-MIDS also allows us to explore the content of the `.dcm`, before being converted to the ORMIR-MIDS structure:

- Load the MESE image in `.dcm`:

In [56]:
img_dcm = load_dicom('data/dicom/Philips_MESE_T2.dcm')

- Explore the `.dcm` image header:

In [ ]:
print(f'Scanner Manufacturer: {img_dcm.bids_header["Manufacturer"]} ({type(img_dcm.bids_header["Manufacturer"])})')
print(f'Scanner Orientation: {img_dcm.orientation} ({type(img_dcm.orientation)})')
print(f'Image Type: {img_dcm.dtype} ({type(img_dcm.dtype)})')
print(f'Image Shape: {img_dcm.shape} ({type(img_dcm.shape)})')
print(f'Scanner Origin: {img_dcm.scanner_origin} ({type(img_dcm.scanner_origin)})')
print(f'Scanner spacing: {img_dcm.pixel_spacing} ({type(img_dcm.pixel_spacing)})')

- Crop the volume. To create a separate subvolume we can do the following. Metadata will be sliced appropriately.

In [ ]:
img_dcm_subvolume = img_dcm[50:90, 50:90, 30:70]
print(f'Original Shape: {img_dcm.shape}')
print(f'Subvolume Shape: {img_dcm_subvolume.shape}')

### 3.2 Convert to SimpleITK

- Since many of us use SimpleITK in our codes, here is a conversion example:

In [ ]:
img_sitk = img_dcm.to_sitk()
print(f'Before conversion, img_dcm is of type: {type(img_dcm)}\nAfter conversion, img_sitk is of type: {type(img_sitk)}')

- Inspect SimpleITK image contents:

In [ ]:
print(img_sitk)

### 3.3 Other conversion methods

Other conversion methods are available, for instance when working with `grayscale`, `nib`, `torch`, or `zarr`:

In [69]:
img_grayscale = img_dcm.to_grayscale()
img_nib = img_dcm.to_nib()
# img_zarr = img_dcm.to_zarr() # %pip install zarr if you want to convert to zarr
# img_torch = img_dcm.to_torch() # %pip install torch if you want to convert to torch

## 4. Visualizing with k3d

In [ ]:
# Extract spacing, origin, and dimensions from a vtkImageData object. We need these to plot the figure in the world coordinates in k3d.
spacing = img_sitk.GetSpacing()
origin = img_sitk.GetOrigin()
dimensions = img_sitk.GetSize()

print(spacing)
print(origin)
print(dimensions)

transform_matrix = np.array([
    [spacing[0] * dimensions[0], 0, 0, origin[0]],
    [0, spacing[1] * dimensions[1], 0, origin[1]],
    [0, 0, spacing[2] * dimensions[2], origin[2]],
    [0, 0, 0, 1]
])

def image2np(image):
    imnp = sitk.GetArrayFromImage(image)
    # Transpose and flip the array to have the same indexing and direction as the original image
    imnp = np.transpose(imnp, (2, 1, 0))
    imnp = np.flip(imnp, axis=2)
    # Enforce float16 type (best for k3d)
    imnp = imnp.astype(np.float16)
    return imnp

imnp_f16 = image2np(img_sitk)
vmin, vmax = np.percentile(imnp_f16, [0.01, 99.99])

transform_s = k3d.transform(custom_matrix=transform_matrix)

# Instantiate the volume plot using k3d-jupyter
plt_volume = k3d.volume(imnp_f16,
                        transform=transform_s,
                        color_map=k3d.colormaps.basic_color_maps.Binary,
                        samples=256,
                        alpha_coef=150,
                        color_range=[vmin, vmax])
plot = k3d.plot()
plot += plt_volume

# Display the plot inline
plot.display()

---
## Dependencies

- For future reproducibility of the workflow:

In [ ]:
%reload_ext watermark

%watermark
%watermark --iversions

---
<a name="ref"></a>
## References

[<sup id="fn1">1</sup>](#fn1-back) Gorgolewski, K., Auer, T., Calhoun, V. et al. The brain imaging data structure, a format for organizing and describing outputs of neuroimaging experiments. Sci Data 3, 160044 (2016). https://doi.org/10.1038/sdata.2016.44

---
## Acknowledgements
ORMIR MIDS is under development by memberes of the [ORMIR](https://www.ormir.org/)
community. 
The development started during the 2nd ORMIR workshop [Sharing and Curating Open Data in Musculoskeletal Imaging Research](https://github.com/ORMIRcommunity/2024_2nd_ORMIR_WS/blob/main/README.md), Zurich, Switzerland. 15-18 January 2024. 
ORMIR MIDS is based on 
[muscle-bids](https://github.com/muscle-bids/muscle-bids), developed during the 1st ORMIR workshop [Building the Jupyter Community in MSK Imaging Research - A Jupyter Commuity Workshop](https://github.com/JCMSK/2022_JCW/blob/main/README.md), Maastricht, The Netherlands, 9-11 June 2022; 
and [ORMIR-PyVoxel](https://github.com/ormir-mids/ormir-pyvoxel) a tool to handle medical image I/O modified from [pyVoxel](https://github.com/pyvoxel/pyvoxel) by Arjun Desai.  


---
<a name="attribution"></a>

Notebook created using the [template](https://github.com/ORMIRcommunity/templates/blob/main/ORMIR_nb_template.ipynb) of the [ORMIR community](https://ormircommunity.github.io/) (version 1.0, 2023)